In [1]:
import pandas as pd
import os
import json
import csv
import re, datetime
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta

from global_functions import *

### Information

Input files:
data_processing/bugs_info.csv
raw_data/bug_history.csv

Output files:
data_processing/bugs_info.csv (TO COMPLETE)
data_processing/bug_history.csv
data/assigned_to_ids.csv

In [2]:
bugs_processing_file = '.'+os.sep+'data_processing'+os.sep+'bugs_info.zip'

bug_history_orig = './raw_data/bug_history.zip'
processed_history_file = './data_processing/bug_history.zip'

injected_assignments_file =  './data/assigned_to_ids.zip'

In [3]:
#Load data
#use os.sep to get the separator based on the operating system so as not to have issues running the script in other operating systems
df_info = pd.read_csv(bugs_processing_file,index_col=False)
df_info['creation_time'] = pd.to_datetime(df_info['creation_time'])
df_info.head(n=2)

,id,release,Product,creation_time,version,resolution,is_assigned,is_resolved,is_fixed,first_assignment_date,last_assignment_date,first_resolved_date,last_resolved_date,first_fixed_date,last_fixed_date
0,475361,4.5,Platform,2015-08-19 10:50:25,4.5,FIXED,0,1,1,NaN,NaN,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z
1,475365,4.6,Platform,2015-08-19 11:34:37,4.6,FIXED,0,1,1,NaN,NaN,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z


In [4]:
len(df_info)

141010

In [5]:
#filter out the history of the bugs we excluded
bug_history = pd.read_csv(bug_history_orig,index_col=False)
bug_history = bug_history[bug_history['id'].isin(df_info.id.unique().tolist())]
bug_history.to_csv(processed_history_file,index=False)

In [6]:
df_bugs = df_info[['id','release','Product','creation_time','version','resolution']].drop_duplicates()

df = bug_history.copy()

#Bugs that were assigned
assigned_bugs = df[(df['what']=='status') & (df['added']=='ASSIGNED')].id.unique().tolist()
#Bugs that were resolved
resolved_bugs = df[(df['what']=='status') & (df['added']=='RESOLVED')].id.unique().tolist()
#Bugs that were fixed
fixed_bugs = df[(df['what']=='resolution') & (df['added']=='FIXED')].id.unique().tolist()


df_bugs['is_assigned'] = 0
df_bugs.loc[df_bugs['id'].isin(assigned_bugs),'is_assigned'] = 1

df_bugs['is_resolved'] = 0
df_bugs.loc[df_bugs['id'].isin(resolved_bugs),'is_resolved'] = 1

df_bugs['is_fixed'] = 0
df_bugs.loc[df_bugs['id'].isin(fixed_bugs),'is_fixed'] = 1

df_bugs.head(n=2)

min_assingment_date = (
    df[(df['id'].isin(assigned_bugs)) & 
       (df['what']=='status') & 
       (df['added']=='ASSIGNED')]
    .groupby('id')
    ['when']
    .min()
    .reset_index()
    .rename(index=str,columns={'when':'first_assignment_date'})
)

df_bugs = pd.merge(df_bugs,min_assingment_date,on=['id'],how='outer')

max_assingment_date = (
    df[(df['id'].isin(assigned_bugs)) & 
       (df['what']=='status') & 
       (df['added']=='ASSIGNED')]
    .groupby('id')
    ['when']
    .max()
    .reset_index()
    .rename(index=str,columns={'when':'last_assignment_date'})
)

df_bugs = pd.merge(df_bugs,max_assingment_date,on=['id'],how='outer')


min_resolved_date = (
    df[(df['id'].isin(resolved_bugs)) & 
      (df['what']=='status') & 
       (df['added']=='RESOLVED')]
    .groupby('id')
    ['when']
    .min()
    .reset_index()
    .rename(index=str,columns={'when':'first_resolved_date'})
)
df_bugs = pd.merge(df_bugs,min_resolved_date,on=['id'],how='outer')

max_resolved_date = (
    df[(df['id'].isin(resolved_bugs)) & 
      (df['what']=='status') & 
       (df['added']=='RESOLVED')]
    .groupby('id')
    ['when']
    .max()
    .reset_index()
    .rename(index=str,columns={'when':'last_resolved_date'})
)
df_bugs = pd.merge(df_bugs,max_resolved_date,on=['id'],how='outer')

min_fixed_date = (
    df[(df['id'].isin(fixed_bugs)) & 
      (df['what']=='resolution') & 
       (df['added']=='FIXED')]
    .groupby('id')
    ['when']
    .min()
    .reset_index()
    .rename(index=str,columns={'when':'first_fixed_date'})
)
df_bugs = pd.merge(df_bugs,min_fixed_date,on=['id'],how='outer')

max_fixed_date = (
    df[(df['id'].isin(fixed_bugs)) & 
      (df['what']=='resolution') & 
       (df['added']=='FIXED')]
    .groupby('id')
    ['when']
    .max()
    .reset_index()
    .rename(index=str,columns={'when':'last_fixed_date'})
)
df_bugs = pd.merge(df_bugs,max_fixed_date,on=['id'],how='outer')

df_bugs = df_bugs.fillna('')
df_bugs.head()

,id,release,Product,creation_time,version,resolution,is_assigned,is_resolved,is_fixed,first_assignment_date,last_assignment_date,first_resolved_date,last_resolved_date,first_fixed_date,last_fixed_date
0,475361,4.5,Platform,2015-08-19 10:50:25,4.5,FIXED,0,1,1,,,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z
1,475365,4.6,Platform,2015-08-19 11:34:37,4.6,FIXED,0,1,1,,,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z
2,475370,4.5,Platform,2015-08-19 12:09:06,4.5,DUPLICATE,0,0,0,,,,,,
3,475379,4.5,Platform,2015-08-19 13:39:12,4.5,,0,0,0,,,,,,
4,475407,4.5,Platform,2015-08-19 17:06:10,4.5,FIXED,0,1,1,,,2015-08-19T17:15:25Z,2015-08-19T17:15:25Z,2015-08-19T17:15:25Z,2015-08-19T17:15:25Z


In [7]:
df_bugs.to_csv(bugs_processing_file,index=False)

### Assignment fixes

In eclipse bug handling, there  are 2 ways that bugs are assigned aside the ASSIGNED status:
1. The bug is assigned to an email address component_name-triaged@eclipse.org
2. The person who is assigned to the bug also resolves it

In these 2 cases, we inject ASSIGNED statuses so that we minimize the threat to validity

#### CASE 1: -triaged@eclipse.org

In [8]:
injected_assignments = pd.DataFrame()

In [9]:
#bugs that are assigned by the status ASSIGNED
bugs_assigned = df_bugs[df_bugs['is_assigned']==1].id.unique().tolist()

triaged_with_email = bug_history[(bug_history['added'].str.find('-triaged@eclipse.org')>=0)]
triaged_with_email_bugs = [x for x in triaged_with_email.id.unique().tolist() if x not in bugs_assigned]
len(triaged_with_email_bugs)

5174

In [10]:
#inject in the history the assignment row
to_inject = triaged_with_email.copy()
to_inject['removed'] = ''
to_inject['who'] = to_inject['added']
to_inject['added'] = 'ASSIGNED'
to_inject['what'] = 'status'

injected_assignments = injected_assignments.append(to_inject,ignore_index=True)

bug_history = bug_history.append(to_inject,ignore_index=True)

df_bugs.loc[df_bugs['id'].isin(to_inject.id.unique().tolist()),'is_assigned'] = 1

#### CASE 2: assinged by email also resolves it

In [11]:
bugs_assigned = df_bugs[df_bugs['is_assigned']==1].id.unique().tolist()
len(bugs_assigned)

52970

In [12]:
history_investigate = bug_history[~bug_history['id'].isin(bugs_assigned)]
history_investigate.head()

,added,id,removed,what,when,who
9,david_audel@fr.ibm.com,100002,jdt-core-inbox@eclipse.org,assigned_to,2006-10-06T19:42:16Z,Olivier_Thomann@ca.ibm.com
10,RESOLVED,100004,NEW,status,2005-06-14T16:32:08Z,dirk_baeumer@ch.ibm.com
11,DUPLICATE,100004,NaN,resolution,2005-06-14T16:32:08Z,dirk_baeumer@ch.ibm.com
43,susan_franklin@us.ibm.com,100014,platform-doc-inbox@eclipse.org,assigned_to,2005-06-23T11:46:04Z,Tod_Creasey@ca.ibm.com
44,RESOLVED,100014,NEW,status,2005-06-23T14:36:36Z,susan@rapicorp.com


In [13]:
#Do the assignment fixes
len(history_investigate[history_investigate['what']=='assigned_to'])

56573

In [14]:
#bugs that have the assigned_to field
assigned_to=history_investigate[history_investigate['what']=='assigned_to'].id.unique()

In [15]:
assigned_to_bugs=df_bugs[df_bugs['id'].isin(assigned_to)]
assigned_resolved=assigned_to_bugs[assigned_to_bugs['is_resolved']>0]
print('# resolved',len(assigned_resolved.id.unique()))
print('# non resolved',len(assigned_to_bugs[assigned_to_bugs['is_resolved']==0]))

# resolved 34341
# non resolved 8126


In [16]:
y=history_investigate[history_investigate['id'].isin(assigned_to)]
y.head()

,added,id,removed,what,when,who
9,david_audel@fr.ibm.com,100002,jdt-core-inbox@eclipse.org,assigned_to,2006-10-06T19:42:16Z,Olivier_Thomann@ca.ibm.com
43,susan_franklin@us.ibm.com,100014,platform-doc-inbox@eclipse.org,assigned_to,2005-06-23T11:46:04Z,Tod_Creasey@ca.ibm.com
44,RESOLVED,100014,NEW,status,2005-06-23T14:36:36Z,susan@rapicorp.com
45,FIXED,100014,NaN,resolution,2005-06-23T14:36:36Z,susan@rapicorp.com
46,3.1 RC4,100014,---,target_milestone,2005-06-23T14:36:36Z,susan@rapicorp.com


In [17]:
#only work with the ones that actually got resolved
y = y[y['id'].isin(assigned_resolved.id.unique().tolist())]
y.head()

,added,id,removed,what,when,who
43,susan_franklin@us.ibm.com,100014,platform-doc-inbox@eclipse.org,assigned_to,2005-06-23T11:46:04Z,Tod_Creasey@ca.ibm.com
44,RESOLVED,100014,NEW,status,2005-06-23T14:36:36Z,susan@rapicorp.com
45,FIXED,100014,NaN,resolution,2005-06-23T14:36:36Z,susan@rapicorp.com
46,3.1 RC4,100014,---,target_milestone,2005-06-23T14:36:36Z,susan@rapicorp.com
47,VERIFIED,100014,RESOLVED,status,2005-06-24T02:33:53Z,susan@rapicorp.com


In [18]:
#find who was assigned to them and who resolved them

In [19]:
resolutions = y[(y['added']=='RESOLVED') & (y['what']=='status')]
resolutions.head()

,added,id,removed,what,when,who
44,RESOLVED,100014,NEW,status,2005-06-23T14:36:36Z,susan@rapicorp.com
69,RESOLVED,100017,NEW,status,2005-11-18T21:19:32Z,john.arthorne@gmail.com
94,RESOLVED,100025,NEW,status,2011-04-25T22:03:02Z,cgold@us.ibm.com
100,RESOLVED,100030,NEW,status,2005-12-07T23:08:28Z,bokowski@google.com
147,RESOLVED,100040,NEW,status,2005-06-15T15:41:25Z,eclipse.felipe@gmail.com


In [20]:
resolutions=resolutions[['id','who']]
resolutions.head()

,id,who
44,100014,susan@rapicorp.com
69,100017,john.arthorne@gmail.com
94,100025,cgold@us.ibm.com
100,100030,bokowski@google.com
147,100040,eclipse.felipe@gmail.com


In [21]:
len(resolutions.id.unique())

34341

In [22]:
assignments = y[(y['what']=='assigned_to')]
assignments.head(n=10)

,added,id,removed,what,when,who
43,susan_franklin@us.ibm.com,100014,platform-doc-inbox@eclipse.org,assigned_to,2005-06-23T11:46:04Z,Tod_Creasey@ca.ibm.com
66,platform-resources-inbox@eclipse.org,100017,jdt-core-inbox@eclipse.org,assigned_to,2005-06-16T11:16:32Z,jerome_lanneluc@fr.ibm.com
87,dejan@ca.ibm.com,100025,Platform-UI-Inbox@eclipse.org,assigned_to,2005-06-14T17:38:37Z,eclipse@pookzilla.net
89,deboer@ca.ibm.com,100025,dejan@ca.ibm.com,assigned_to,2005-06-14T18:07:12Z,dejan@ca.ibm.com
92,platform-ua-inbox@eclipse.org,100025,deboer@ca.ibm.com,assigned_to,2009-04-15T14:37:25Z,cgold@us.ibm.com
99,bokowski@ca.ibm.com,100030,Platform-UI-Inbox@eclipse.org,assigned_to,2005-06-14T18:41:44Z,eclipse@pookzilla.net
146,felipe_heidrich@ca.ibm.com,100040,platform-swt-inbox@eclipse.org,assigned_to,2005-06-14T19:24:25Z,eclipse.felipe@gmail.com
225,platform-doc-inbox@eclipse.org,100066,platform-help-inbox@eclipse.org,assigned_to,2005-06-14T20:18:14Z,sonia_dimitrov@ca.ibm.com
271,veronika_irvine@ca.ibm.com,100083,platform-swt-inbox@eclipse.org,assigned_to,2005-06-15T18:29:23Z,snorthov@ca.ibm.com
343,platform-ant-inbox@eclipse.org,100101,jdt-core-inbox@eclipse.org,assigned_to,2005-06-14T23:04:54Z,philippe_mulet@fr.ibm.com


In [23]:
assignments = assignments[['id','added']].rename(index=str,columns={'added':'who'})
assignments.head()

,id,who
43,100014,susan_franklin@us.ibm.com
66,100017,platform-resources-inbox@eclipse.org
87,100025,dejan@ca.ibm.com
89,100025,deboer@ca.ibm.com
92,100025,platform-ua-inbox@eclipse.org


In [24]:
len(assignments.id.unique())

34341

In [25]:
merged_assign_res_bugs = pd.merge(resolutions,assignments,on=['id','who'],how='inner')
merged_assign_res_bugs

,id,who
0,239496,rafael.oliveira@gmail.com
1,241551,jerome_lanneluc@fr.ibm.com
2,244176,francisu@ieee.org
3,246493,gheorghe@ca.ibm.com
4,247541,canada.eric@gmail.com
5,254343,pwebster@ca.ibm.com
6,254495,grant_gayed@ca.ibm.com
7,254496,grant_gayed@ca.ibm.com
8,254497,Silenio_Quarti@ca.ibm.com
9,254813,Kevin_McGuire@ca.ibm.com


In [26]:
len(merged_assign_res_bugs.id.unique())

230

In [27]:
#inject the assignment for the 230 cases where the person who got assinged the bug, also resolved it

In [28]:
assigned_injections = y[(y['what']=='assigned_to')]
assigned_injections = assigned_injections[assigned_injections['id'].isin(merged_assign_res_bugs.id.unique())]

#inject in the history the assignment row
assigned_injections['removed'] = ''
assigned_injections['who'] = assigned_injections['added']
assigned_injections['what'] = 'status'
assigned_injections['added'] = 'ASSIGNED'

injected_assignments = injected_assignments.append(assigned_injections,ignore_index=True)

bug_history = bug_history.append(assigned_injections,ignore_index=True)

df_bugs.loc[df_bugs['id'].isin(assigned_injections.id.unique().tolist()),'is_assigned'] = 1


In [29]:
injected_assignments.to_csv(injected_assignments_file,index=False)

In [30]:
#Re-run the info analysis after the injection
df_bugs = df_info[['id','release','Product','creation_time','version','resolution']].drop_duplicates()

df = bug_history.copy()

#Bugs that were assigned
assigned_bugs = df[(df['what']=='status') & (df['added']=='ASSIGNED')].id.unique().tolist()
#Bugs that were resolved
resolved_bugs = df[(df['what']=='status') & (df['added']=='RESOLVED')].id.unique().tolist()
#Bugs that were fixed
fixed_bugs = df[(df['what']=='resolution') & (df['added']=='FIXED')].id.unique().tolist()


df_bugs['is_assigned'] = 0
df_bugs.loc[df_bugs['id'].isin(assigned_bugs),'is_assigned'] = 1

df_bugs['is_resolved'] = 0
df_bugs.loc[df_bugs['id'].isin(resolved_bugs),'is_resolved'] = 1

df_bugs['is_fixed'] = 0
df_bugs.loc[df_bugs['id'].isin(fixed_bugs),'is_fixed'] = 1

df_bugs.head(n=2)

min_assingment_date = (
    df[(df['id'].isin(assigned_bugs)) & 
       (df['what']=='status') & 
       (df['added']=='ASSIGNED')]
    .groupby('id')
    ['when']
    .min()
    .reset_index()
    .rename(index=str,columns={'when':'first_assignment_date'})
)

df_bugs = pd.merge(df_bugs,min_assingment_date,on=['id'],how='outer')

max_assingment_date = (
    df[(df['id'].isin(assigned_bugs)) & 
       (df['what']=='status') & 
       (df['added']=='ASSIGNED')]
    .groupby('id')
    ['when']
    .max()
    .reset_index()
    .rename(index=str,columns={'when':'last_assignment_date'})
)

df_bugs = pd.merge(df_bugs,max_assingment_date,on=['id'],how='outer')


min_resolved_date = (
    df[(df['id'].isin(resolved_bugs)) & 
      (df['what']=='status') & 
       (df['added']=='RESOLVED')]
    .groupby('id')
    ['when']
    .min()
    .reset_index()
    .rename(index=str,columns={'when':'first_resolved_date'})
)
df_bugs = pd.merge(df_bugs,min_resolved_date,on=['id'],how='outer')

max_resolved_date = (
    df[(df['id'].isin(resolved_bugs)) & 
      (df['what']=='status') & 
       (df['added']=='RESOLVED')]
    .groupby('id')
    ['when']
    .max()
    .reset_index()
    .rename(index=str,columns={'when':'last_resolved_date'})
)
df_bugs = pd.merge(df_bugs,max_resolved_date,on=['id'],how='outer')

min_fixed_date = (
    df[(df['id'].isin(fixed_bugs)) & 
      (df['what']=='resolution') & 
       (df['added']=='FIXED')]
    .groupby('id')
    ['when']
    .min()
    .reset_index()
    .rename(index=str,columns={'when':'first_fixed_date'})
)
df_bugs = pd.merge(df_bugs,min_fixed_date,on=['id'],how='outer')

max_fixed_date = (
    df[(df['id'].isin(fixed_bugs)) & 
      (df['what']=='resolution') & 
       (df['added']=='FIXED')]
    .groupby('id')
    ['when']
    .max()
    .reset_index()
    .rename(index=str,columns={'when':'last_fixed_date'})
)
df_bugs = pd.merge(df_bugs,max_fixed_date,on=['id'],how='outer')

df_bugs = df_bugs.fillna('')
df_bugs.head()

,id,release,Product,creation_time,version,resolution,is_assigned,is_resolved,is_fixed,first_assignment_date,last_assignment_date,first_resolved_date,last_resolved_date,first_fixed_date,last_fixed_date
0,475361,4.5,Platform,2015-08-19 10:50:25,4.5,FIXED,0,1,1,,,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z,2015-08-19T11:53:03Z
1,475365,4.6,Platform,2015-08-19 11:34:37,4.6,FIXED,0,1,1,,,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z,2015-08-26T14:29:30Z
2,475370,4.5,Platform,2015-08-19 12:09:06,4.5,DUPLICATE,0,0,0,,,,,,
3,475379,4.5,Platform,2015-08-19 13:39:12,4.5,,0,0,0,,,,,,
4,475407,4.5,Platform,2015-08-19 17:06:10,4.5,FIXED,0,1,1,,,2015-08-19T17:15:25Z,2015-08-19T17:15:25Z,2015-08-19T17:15:25Z,2015-08-19T17:15:25Z


In [31]:
print('Injected assignments for bugs:'+str(len(injected_assignments.id.unique())))

Injected assignments for bugs:6811


In [33]:
bug_history.to_csv(processed_history_file,index=False)
df_bugs.to_csv(bugs_processing_file,index=False)